In [1]:
name = "CombineSignals"
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [9]:
alphas = [
    "UserItemBiases",
    "MatrixFactorization10",
    "MatrixFactorization20",
    "MatrixFactorization40",
    "ItemCF.16",
    "ItemCF.64",
    "ItemCF.256",
    "ItemCF.1024",
];

In [10]:
function evaluate(X, y, β)
    y_pred = X * β
    y_pred = clamp.(y_pred, 1, 10)
    rmse(y, y_pred), mae(y, y_pred), r2(y, y_pred), mse(y, y_pred)
end;

In [11]:
function get_dep(split)
    return get_split(split).rating
end

function get_indep(split, alphas)
    users = get_split(split).user
    X = zeros(length(users), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], split).rating
    end
    X
end;

In [12]:
β = get_indep("validation", alphas) \ get_dep("validation")

7-element Vector{Float64}:
  0.9944237538411016
  0.1291942937061494
  0.1421369036591969
  0.19194364899923477
  0.18553389207088772
 -0.046876241875641614
  0.5504212539839372

In [7]:
evaluate(get_indep("validation", alphas), get_dep("validation"), β)

(1.1025944225694135, 0.7998772497204256, 0.616561899248498, 1.2157144606811783)

In [8]:
evaluate(get_indep("test", alphas), get_dep("test"), β)

(1.1027292962553725, 0.7999783483150411, 0.616536954272277, 1.216011900819869)

## TODO store Coefs